In [40]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
import pandas as pd
from scipy.spatial.distance import cdist
import numpy as np

# Product_df

In [41]:
product_df = pd.read_csv('processed_product_final.csv')
product_df = product_df.iloc[: , 1:]
product_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,combined_info,embeddings
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,Product Name: Turtle Check Men Navy Blue Shirt...,[-2.68268585e-02 6.60445839e-02 1.56628564e-...
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,Product Name: Peter England Men Party Blue Jea...,[-3.63447741e-02 -1.79072991e-02 1.01005035e-...
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,"Product Name: Titan Women Silver Watch, Gender...",[-8.63776952e-02 -4.10641469e-02 3.33572179e-...
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,Product Name: Manchester United Men Solid Blac...,[-4.90391180e-02 -1.59678720e-02 -3.54683548e-...
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,"Product Name: Puma Men Grey T-shirt, Gender: M...",[-2.36744732e-02 4.10940349e-02 -2.29127221e-...


In [42]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44446 entries, 0 to 44445
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44446 non-null  int64  
 1   gender              44446 non-null  object 
 2   masterCategory      44446 non-null  object 
 3   subCategory         44446 non-null  object 
 4   articleType         44446 non-null  object 
 5   baseColour          44431 non-null  object 
 6   season              44425 non-null  object 
 7   year                44445 non-null  float64
 8   usage               44129 non-null  object 
 9   productDisplayName  44439 non-null  object 
 10  combined_info       44446 non-null  object 
 11  embeddings          44446 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 4.1+ MB


# Customer_df

In [43]:
customer_df = pd.read_csv('processed_customer_final.csv', encoding='utf-8')
customer_df = customer_df.iloc[: , 1:]
customer_df.head()

,customer_id,first_name,last_name,username,email,gender,birthdate,device_type,device_id,device_version,home_location,home_country,first_join_date,age,past_trans
0,2870,Lala,Maryati,671a0865-ac4e-4dc4-9c4f-c286a1176f7e,671a0865_ac4e_4dc4_9c4f_c286a1176f7e@startupca...,Women,14/06/1996,iOS,c9c0de76-0a6c-4ac2-843f-65264ab9fe63,iPhone; CPU iPhone OS 14_2_1 like Mac OS X,Sumatera Barat,Indonesia,21/07/2019,27,[]
1,8193,Maimunah,Laksmiwati,83be2ba7-8133-48a4-bbcb-b46a2762473f,83be2ba7_8133_48a4_bbcb_b46a2762473f@zakyfound...,Women,16/08/1993,Android,fb331c3d-f42e-40fe-afe2-b4b73a8a6e25,Android 2.2.1,Jakarta Raya,Indonesia,16/07/2017,30,[]
2,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,Men,23/01/1989,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,Nusa Tenggara Barat,Indonesia,23/08/2020,35,"['Lino Perros Women Solid Brown Belt', ""Puma W..."
3,88813,Cahyadi,Maheswara,df797edf-b465-4a80-973b-9fbb612260c2,df797edf_b465_4a80_973b_9fbb612260c2@zakyfound...,Men,05/01/1991,iOS,f4c18515-c5be-419f-8142-f037be47c9cd,iPad; CPU iPad OS 14_2 like Mac OS X,Kalimantan Timur,Indonesia,03/10/2021,33,[]
4,82542,Irnanto,Wijaya,36ab08e1-03de-42a8-9e3b-59528c798824,36ab08e1_03de_42a8_9e3b_59528c798824@startupca...,Men,15/07/2000,iOS,e46e4c36-4630-4736-8fcf-663db29ca3b0,iPhone; CPU iPhone OS 10_3_3 like Mac OS X,Kalimantan Selatan,Indonesia,11/04/2021,23,['Arrow Woman Lana White Shirt']


In [44]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   customer_id      100000 non-null  int64 
 1   first_name       100000 non-null  object
 2   last_name        100000 non-null  object
 3   username         100000 non-null  object
 4   email            100000 non-null  object
 5   gender           100000 non-null  object
 6   birthdate        100000 non-null  object
 7   device_type      100000 non-null  object
 8   device_id        100000 non-null  object
 9   device_version   100000 non-null  object
 10  home_location    100000 non-null  object
 11  home_country     100000 non-null  object
 12  first_join_date  100000 non-null  object
 13  age              100000 non-null  int64 
 14  past_trans       100000 non-null  object
dtypes: int64(2), object(13)
memory usage: 11.4+ MB


In [45]:
product_df['embeddings'].head()

,embeddings
0,[-2.68268585e-02 6.60445839e-02 1.56628564e-...
1,[-3.63447741e-02 -1.79072991e-02 1.01005035e-...
2,[-8.63776952e-02 -4.10641469e-02 3.33572179e-...
3,[-4.90391180e-02 -1.59678720e-02 -3.54683548e-...
4,[-2.36744732e-02 4.10940349e-02 -2.29127221e-...


# Product Similarity Matrix

In [46]:
! pip install faiss-gpu

In [47]:
import numpy as np
import pandas as pd
import ast
import re
import faiss


# Step 1: Convert embeddings from string to numeric array
# Precompile regex to replace whitespaces between numeric values with commas
comma_insert_pattern = re.compile(r'(?<=\d)\s+(?=-?\d)')

def parse_embedding(embedding_str):
    # Assuming the embedding string contains numeric values separated by spaces or commas
    try:
        # Replace spaces between numbers with commas if needed
        formatted_str = comma_insert_pattern.sub(',', embedding_str.strip())
        # Convert formatted string into a list and then to float32
        return np.array(ast.literal_eval(formatted_str), dtype=np.float32)
    except Exception as e:
        # Handle parsing failure gracefully by providing a default embedding vector
        print(f"Failed to parse: {embedding_str[:50]}... Error: {e}")
        return np.zeros((768,), dtype=np.float32)  # Assuming 768 dimensions for failure

# Apply the function to parse embeddings
product_df['embeddings'] = product_df['embeddings'].apply(parse_embedding)

# Step 2: Stack embeddings into a 2D array with float32 data type
try:
    product_embeddings = np.stack(product_df['embeddings'].values).astype(np.float32)
except ValueError as e:
    print("Failed to stack embeddings. Ensure all embeddings have the same dimension.")
    raise e

# Step 3: Use Faiss for Efficient Similarity Search
d = product_embeddings.shape[1]  # Dimensionality of embeddings
index = faiss.IndexFlatL2(d)  # Use L2 distance for similarity

# Add embeddings to the Faiss index
index.add(product_embeddings)

# Search for top 10 similar items for each product
k = 10
distances, indices = index.search(product_embeddings, k)

# Print the shape of the results to confirm
print(f"Shape of distances: {distances.shape}, Shape of indices: {indices.shape}")

# Step 4: Create a DataFrame with similarity information for each product
similarity_results = []

for i, (dist_row, ind_row) in enumerate(zip(distances, indices)):
    product_id = product_df.iloc[i]['id']
    for dist, ind in zip(dist_row, ind_row):
        similar_product_id = product_df.iloc[ind]['id']
        similarity_results.append({
            'product_id': product_id,
            'similar_product_id': similar_product_id,
            'distance': dist
        })

similarity_df = pd.DataFrame(similarity_results)

# Print a sample of the similarity DataFrame
#print(similarity_df.head())


Shape of distances: (44446, 10), Shape of indices: (44446, 10)


In [48]:
# Save the similarity DataFrame to a CSV file
similarity_df.to_csv('product_similarity_matrix.csv', index=False)

import pickle
# Alternatively, save as a pickle for faster loading
similarity_df.to_pickle('product_similarity_matrix.pkl')

In [49]:
# Load the similarity DataFrame from the CSV file
similarity_df = pd.read_csv('product_similarity_matrix.csv')

# Or load from the pickle file
similarity_df = pd.read_pickle('product_similarity_matrix.pkl')

In [50]:
# Run an Experiment for Product ID
product_id = 15970

# Check if the product ID exists in the DataFrame
if product_id in similarity_df['product_id'].values:
    # Filter out the rows with the given product_id
    similar_products = similarity_df[similarity_df['product_id'] == product_id]

    # Sort by distance to get the most similar products first (lower distance is more similar)
    similar_products = similar_products.sort_values(by='distance')

    # Print the top 10 similar products
    print(f"Top 10 similar products to product ID {product_id}:")
    print(similar_products.head(10))
else:
    print(f"Product ID {product_id} not found in the similarity data.")

Top 10 similar products to product ID 15970:
   product_id  similar_product_id  distance
0       15970               15970  0.000000
1       15970               26396  0.049105
2       15970               15996  0.069805
3       15970               15994  0.069805
4       15970               15992  0.069805
5       15970               15972  0.090451
6       15970               15993  0.112898
7       15970               15974  0.127551
8       15970               15981  0.133185
9       15970               15968  0.133185


# User Similarity Matrix

In [56]:
# Step 1: Convert `past_trans` strings to lists
def parse_past_transactions(past_trans_str):
    if isinstance(past_trans_str, str) and past_trans_str.strip():
        try:
            # Clean up potential issues in the string by removing unwanted characters
            # Retain only alphanumeric characters and certain symbols
            cleaned_str = re.sub(r'[^a-zA-Z0-9\s,\[\]\(\)\{\}\-\'"]', '', past_trans_str)

            # Replace single quotes with double quotes for JSON compatibility
            cleaned_str = cleaned_str.replace("'", '"')

            # Remove multiple spaces to prevent parsing issues
            cleaned_str = re.sub(r'\s+', ' ', cleaned_str)

            # Ensure the cleaned string is formatted properly as a list
            if not (cleaned_str.startswith('[') and cleaned_str.endswith(']')):
                cleaned_str = f"[{cleaned_str}]"

            # Try using JSON parsing first (safer)
            try:
                return json.loads(cleaned_str)
            except json.JSONDecodeError:
                # Fallback to ast.literal_eval if JSON parsing fails
                return ast.literal_eval(cleaned_str)

        except (ValueError, SyntaxError) as e:
            print(f"Failed to parse: {past_trans_str[:50]}... Error: {e}")
            return []
    elif isinstance(past_trans_str, list):
        # If it's already a list, just return it
        return past_trans_str
    else:
        # For NaN or other types, return an empty list
        return []

# Assume `customer_df` is your DataFrame
# Apply the function to parse `past_trans`
customer_df['past_trans'] = customer_df['past_trans'].apply(parse_past_transactions)

In [57]:
customer_df['past_trans'].head(20)

,past_trans
0,[]
1,[]
2,[]
3,[]
4,[Arrow Woman Lana White Shirt]
5,[]
6,[]
7,"[Spykar Men Check Red Shirts, Puma Men Trainin..."
8,"[Wildcraft Lavender Grey Slingbag, Folklore Wo..."
9,[]


In [58]:
# Check how many rows are not successfully parsed
not_parsed_count = (~are_all_parsed).sum()

# If any rows are not successfully parsed, print the corresponding rows
if not_parsed_count > 0:
    print(f"Number of rows not successfully parsed: {not_parsed_count}")
    print("Rows that failed to parse:")
    print(customer_df[~are_all_parsed])
else:
    print("All rows were successfully parsed as lists.")

All rows were successfully parsed as lists.


In [54]:
t

NameError: name 't' is not defined

In [59]:
# Step 2: Create User-Item Interaction Matrix (Sparse Representation)
from scipy.sparse import csr_matrix
# Get all unique product IDs from past transactions
all_products = set()
customer_df['past_trans'].apply(lambda x: all_products.update(x))

# Create a mapping from product ID to column index
product_to_index = {product: idx for idx, product in enumerate(all_products)}

# Initialize lists to construct a sparse matrix
row_indices = []
col_indices = []
data = []

# Populate the data for the sparse user-item matrix
for user_idx, past_trans in enumerate(customer_df['past_trans']):
    for product in past_trans:
        if product in product_to_index:
            product_idx = product_to_index[product]
            row_indices.append(user_idx)
            col_indices.append(product_idx)
            data.append(1)

# Create a sparse user-item interaction matrix
n_users = customer_df.shape[0]
n_products = len(all_products)
user_item_sparse_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(n_users, n_products))

In [60]:
# Print some details about the matrix
print("Shape of User-Item Interaction Matrix:", user_item_sparse_matrix.shape)
print("Number of non-zero entries (interactions):", user_item_sparse_matrix.nnz)

# Convert a small portion of the matrix to dense format for display
# Display interactions for the first 5 users and first 10 products
sample_matrix = user_item_sparse_matrix[:5, :10].toarray()

# Convert to DataFrame for better readability
sample_df = pd.DataFrame(sample_matrix, columns=[f"Product_{i}" for i in range(10)])
sample_df.index = [f"User_{i}" for i in range(5)]

print("\nSample of User-Item Interaction Matrix (first 5 users and 10 products):")
print(sample_df)

Shape of User-Item Interaction Matrix: (100000, 26736)
Number of non-zero entries (interactions): 124874

Sample of User-Item Interaction Matrix (first 5 users and 10 products):
        Product_0  Product_1  Product_2  Product_3  Product_4  Product_5  \
User_0          0          0          0          0          0          0   
User_1          0          0          0          0          0          0   
User_2          0          0          0          0          0          0   
User_3          0          0          0          0          0          0   
User_4          0          0          0          0          0          0   

        Product_6  Product_7  Product_8  Product_9  
User_0          0          0          0          0  
User_1          0          0          0          0  
User_2          0          0          0          0  
User_3          0          0          0          0  
User_4          0          0          0          0  


User-User Similarity Matrix: This matrix represents the similarity between users based on their item interactions. It is key for user-based collaborative filtering, where recommendations are made by finding users with similar interaction patterns.

In [61]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix, vstack
from scipy.sparse import save_npz


# Step 3: Calculate User-User Similarity Matrix in Smaller Batches
num_users = user_item_sparse_matrix.shape[0]
batch_size = 100  # Small batch size to avoid kernel crash

# Initialize an empty list to store similarity results in chunks
similarity_rows = []

for start in range(0, num_users, batch_size):
    end = min(start + batch_size, num_users)
    print(f"Processing batch from user {start} to user {end}")

    # Calculate similarity for the current batch against all users (in sparse form)
    batch_similarity = cosine_similarity(user_item_sparse_matrix[start:end], user_item_sparse_matrix, dense_output=False)
    similarity_rows.append(csr_matrix(batch_similarity))  # Convert to CSR format if not already

# Stack all the batches to form the complete user similarity matrix (still sparse)
user_similarity_matrix_sparse = vstack(similarity_rows)

# Optional: Save the similarity matrix for later use
# You can use scipy's save_npz method to save sparse matrices
save_npz('user_similarity_matrix.npz', user_similarity_matrix_sparse)


Processing batch from user 0 to user 100
Processing batch from user 100 to user 200
Processing batch from user 200 to user 300
Processing batch from user 300 to user 400
Processing batch from user 400 to user 500
Processing batch from user 500 to user 600
Processing batch from user 600 to user 700
Processing batch from user 700 to user 800
Processing batch from user 800 to user 900
Processing batch from user 900 to user 1000
Processing batch from user 1000 to user 1100
Processing batch from user 1100 to user 1200
Processing batch from user 1200 to user 1300
Processing batch from user 1300 to user 1400
Processing batch from user 1400 to user 1500
Processing batch from user 1500 to user 1600
Processing batch from user 1600 to user 1700
Processing batch from user 1700 to user 1800
Processing batch from user 1800 to user 1900
Processing batch from user 1900 to user 2000
Processing batch from user 2000 to user 2100
Processing batch from user 2100 to user 2200
Processing batch from user 2200

In [63]:
from scipy.sparse import load_npz

# Load the saved user similarity matrix (assuming it's stored in sparse format)
user_similarity_matrix_sparse = load_npz('user_similarity_matrix.npz')

# Example: Find similar users for a specific user ID
user_id = 88813

if user_id in range(user_similarity_matrix_sparse.shape[0]):
    # Convert the sparse row to a dense array and create a DataFrame for easier manipulation
    user_similarity_df = pd.DataFrame(user_similarity_matrix_sparse.toarray(), index=range(user_similarity_matrix_sparse.shape[0]))

    similar_users = user_similarity_df.loc[user_id].sort_values(ascending=False).drop(user_id).head(10)
    print(f"Top 10 similar users to user {user_id}:\n{similar_users}")
else:
    print(f"User {user_id} not found in the similarity matrix.")


Top 10 similar users to user 88813:
91476    0.707107
59800    0.500000
74779    0.408248
82316    0.316228
94527    0.288675
85429    0.250000
12301    0.144338
73753    0.107833
66669    0.000000
66668    0.000000
Name: 88813, dtype: float64


In [64]:
# Example: Find similar users for a specific user ID
user_id = 1000

if user_id in range(user_similarity_matrix_sparse.shape[0]):
    # Convert the sparse row to a dense array and create a DataFrame for easier manipulation
    user_similarity_df = pd.DataFrame(user_similarity_matrix_sparse.toarray(), index=range(user_similarity_matrix_sparse.shape[0]))

    similar_users = user_similarity_df.loc[user_id].sort_values(ascending=False).drop(user_id).head(10)
    print(f"Top 10 similar users to user {user_id}:\n{similar_users}")
else:
    print(f"User {user_id} not found in the similarity matrix.")

Top 10 similar users to user 1000:
41406    0.500000
78385    0.500000
84295    0.500000
13385    0.353553
2291     0.353553
62907    0.353553
5064     0.353553
84053    0.288675
83762    0.288675
74968    0.288675
Name: 1000, dtype: float64


In [66]:
# Example: Find similar users for a specific user ID
user_id = 2870

if user_id in range(user_similarity_matrix_sparse.shape[0]):
    # Convert the sparse row to a dense array and create a DataFrame for easier manipulation
    user_similarity_df = pd.DataFrame(user_similarity_matrix_sparse.toarray(), index=range(user_similarity_matrix_sparse.shape[0]))

    similar_users = user_similarity_df.loc[user_id].sort_values(ascending=False).drop(user_id).head(10)
    print(f"Top 10 similar users to user {user_id}:\n{similar_users}")
else:
    print(f"User {user_id} not found in the similarity matrix.")

Top 10 similar users to user 2870:
58978    0.707107
93607    0.500000
75443    0.408248
66671    0.000000
66669    0.000000
66668    0.000000
66667    0.000000
66666    0.000000
66665    0.000000
66664    0.000000
Name: 2870, dtype: float64


# Hybrid Recommendation System
### Content Based Filtering & Collaborative Filtering

with Additional Handling

No Liked Products:
If the user has no liked products, we immediately provide the top popular seasonal products based on frequency in product_df.

No Seasonal Products Based on Preferences:
After attempting collaborative and content-based filtering, if no relevant seasonal products are found, we fallback to popular items for the current season.

Final Check:
If the recommendations list is still empty by the end, a final fallback provides the top popular seasonal products.

In [ ]:
import numpy as np
import pandas as pd
import ast
from scipy.sparse import load_npz

# Assuming customer_df, product_df, similarity_df, and user_similarity_matrix_sparse are already defined.

# Load the user similarity matrix
user_similarity_matrix_sparse = load_npz('user_similarity_matrix.npz')
user_similarity_df = pd.DataFrame(user_similarity_matrix_sparse.toarray(), index=customer_df['customer_id'], columns=customer_df['customer_id'])

# Function to determine the current season
def determine_season():
    # For simplicity, we return a fixed value for the season.
    # You can replace this logic with a real-time approach if needed.
    return 'Summer'

# Function to get similar users for a given user_id
def get_similar_users(user_id, top_n=5):
    if user_id in user_similarity_df.index:
        similar_users = user_similarity_df.loc[user_id].sort_values(ascending=False).drop(user_id).head(top_n).index.tolist()
        return similar_users
    else:
        print(f"User {user_id} not found in the similarity matrix.")
        return []

# Function to get hybrid recommendations for a user
def get_recommendations(user_id, top_n=10):
    recommendations = []

    # Get the user's liked products
    user_data = customer_df[customer_df['customer_id'] == user_id]
    if user_data.empty:
        print(f"User {user_id} not found in customer_df. Returning empty recommendations.")
        return recommendations  # Return empty if user not found

    liked_products = user_data['past_trans'].values[0]
    liked_products = ast.literal_eval(liked_products) if isinstance(liked_products, str) else liked_products

    # If no liked products are found, provide a default list of popular seasonal products
    current_season = determine_season()
    if not liked_products:
        print(f"No liked products found for user {user_id}. Returning popular seasonal products.")
        seasonal_popular = product_df[product_df['season'] == current_season]['id'].value_counts().index.tolist()

        # Assign exponential decay scores based on rank (higher rank gets higher score)
        total_products = len(seasonal_popular)
        decay_factor = 0.9  # Adjust decay factor to control how fast the score decreases
        recommendations = [
            {
                'customer_id': user_id,
                'product_id': product,
                'rank_score': decay_factor ** idx  # Exponentially decaying score
            }
            for idx, product in enumerate(seasonal_popular[:top_n])
        ]

        return recommendations

    print(f"Liked products for user {user_id}: {liked_products}")

    # Get similar users
    similar_users = get_similar_users(user_id, top_n=5)
    similar_users_liked_products = []
    similar_users_scores = []

    if similar_users:
        # Get products liked by similar users and their corresponding similarity scores
        for similar_user in similar_users:
            similarity_score = user_similarity_df.loc[user_id, similar_user]
            similar_user_products = customer_df[customer_df['customer_id'] == similar_user]['past_trans'].values[0]
            similar_user_products = ast.literal_eval(similar_user_products) if isinstance(similar_user_products, str) else similar_user_products
            similar_users_liked_products.extend(similar_user_products)
            similar_users_scores.extend([similarity_score] * len(similar_user_products))

    print(f"Products liked by similar users for user {user_id}: {similar_users_liked_products}")

    # Get products similar to what the user liked
    similar_products = similarity_df[similarity_df['product_id'].isin(liked_products)]
    similar_products_ids = similar_products['similar_product_id'].tolist()
    similar_products_scores = similar_products['distance'].tolist()  # Assuming smaller distance means higher similarity

    print(f"Products similar to liked products for user {user_id}: {similar_products_ids}")

    # Determine the current season
    print(f"Current season: {current_season}")

    # Filter recommendations by season
    seasonal_products = product_df[product_df['id'].isin(similar_products_ids + similar_users_liked_products) &
                                   (product_df['season'] == current_season)]

    if seasonal_products.empty:
        print(f"No seasonal products found for user {user_id}'s preferences. Returning popular items in season.")
        seasonal_products = product_df[product_df['season'] == current_season].head(top_n)

    # Aggregate scores for products
    product_scores = {}

    # Add scores from similar users' liked products
    for product_id, score in zip(similar_users_liked_products, similar_users_scores):
        if product_id in seasonal_products['id'].values:
            product_scores[product_id] = product_scores.get(product_id, 0) + score

    # Add scores from similar products
    for product_id, score in zip(similar_products_ids, similar_products_scores):
        if product_id in seasonal_products['id'].values:
            # Using inverse distance as a similarity score (since smaller distance means higher similarity)
            similarity_score = 1 / (1 + score)
            product_scores[product_id] = product_scores.get(product_id, 0) + similarity_score

    # Normalize the scores to be between 0 and 1
    max_score = max(product_scores.values()) if product_scores else 1
    for product_id in product_scores:
        product_scores[product_id] /= max_score

    # Sort products by their aggregated scores in descending order and select the top N
    ranked_products = sorted(product_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]

    # Prepare the recommendation list for the user
    recommendations = [{'customer_id': user_id, 'product_id': product_id, 'rank_score': rank_score}
                       for product_id, rank_score in ranked_products]

    # Check if recommendations list is still empty, and if so, use fallback seasonal popular items
    if not recommendations:
        print(f"No recommendations found based on preferences for user {user_id}. Using popular seasonal products.")
        seasonal_popular = product_df[product_df['season'] == current_season]['id'].value_counts().index.tolist()

        # Assign exponential decay scores based on rank (higher rank gets higher score)
        total_products = len(seasonal_popular)
        decay_factor = 0.9  # Adjust decay factor to control how fast the score decreases
        recommendations = [
            {
                'customer_id': user_id,
                'product_id': product,
                'rank_score': decay_factor ** idx  # Exponentially decaying score
            }
            for idx, product in enumerate(seasonal_popular[:top_n])
        ]

    # Convert recommendations to DataFrame and save to CSV
    recommendations_df = pd.DataFrame(recommendations)
    csv_filename = f'{user_id}_user_recommendations_{current_season}.csv'
    recommendations_df.to_csv(csv_filename, index=False)

    print(f"Recommendations for user {user_id} saved to {csv_filename}.")
    print("Generated Recommendations:", recommendations)

    return recommendations


In [ ]:
import requests
import numpy as np
import pandas as pd
import ast
from scipy.sparse import load_npz

# Load the user similarity matrix
user_similarity_matrix_sparse = load_npz('user_similarity_matrix.npz')
user_similarity_df = pd.DataFrame(user_similarity_matrix_sparse.toarray(), index=customer_df['customer_id'], columns=customer_df['customer_id'])

# Function to determine the current season based on real-time weather and location
def determine_season():
    # Set up API keys (replace with your actual keys)
    ipinfo_api_key = "396370bd35a8161a2935f7cb0bd378a9"  # positionstack.com
    weather_api_key = "5d8f488d96447f74ae5a04a7a41eaff3" # openweathermap.org
    
    # Step 1: Get the user's approximate location using IPinfo
    location_url = f"https://ipinfo.io?token={ipinfo_api_key}"
    location_response = requests.get(location_url)
    location_data = location_response.json()
    
    if 'loc' not in location_data:
        print("Could not determine location. Defaulting to 'Summer'.")
        return 'Summer'  # Default season if location fetch fails

    latitude, longitude = map(float, location_data['loc'].split(','))

    # Step 2: Get real-time weather data using OpenWeatherMap
    weather_url = f"http://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={weather_api_key}"
    weather_response = requests.get(weather_url)
    weather_data = weather_response.json()

    # Extract temperature in Celsius
    temperature_k = weather_data['main']['temp']
    temperature_c = temperature_k - 273.15

    # Basic logic to determine season based on temperature
    if temperature_c > 25:
        season = 'Summer'
    elif 15 <= temperature_c <= 25:
        season = 'Spring' if weather_data['weather'][0]['main'] in ['Clear', 'Rain'] else 'Fall'
    else:
        season = 'Winter'

    print(f"Determined season based on real-time data: {season}")
    return season

# Function to get similar users for a given user_id
def get_similar_users(user_id, top_n=5):
    if user_id in user_similarity_df.index:
        similar_users = user_similarity_df.loc[user_id].sort_values(ascending=False).drop(user_id).head(top_n).index.tolist()
        return similar_users
    else:
        print(f"User {user_id} not found in the similarity matrix.")
        return []

# Function to get hybrid recommendations for a user
def get_recommendations(user_id, top_n=10):
    recommendations = []

    # Get the user's liked products
    user_data = customer_df[customer_df['customer_id'] == user_id]
    if user_data.empty:
        print(f"User {user_id} not found in customer_df. Returning empty recommendations.")
        return recommendations  # Return empty if user not found

    liked_products = user_data['past_trans'].values[0]
    liked_products = ast.literal_eval(liked_products) if isinstance(liked_products, str) else liked_products

    # If no liked products are found, provide a default list of popular seasonal products
    current_season = determine_season()
    if not liked_products:
        print(f"No liked products found for user {user_id}. Returning popular seasonal products.")
        seasonal_popular = product_df[product_df['season'] == current_season]['id'].value_counts().index.tolist()

        # Assign exponential decay scores based on rank (higher rank gets higher score)
        decay_factor = 0.9
        recommendations = [
            {'customer_id': user_id, 'product_id': product, 'rank_score': decay_factor ** idx}
            for idx, product in enumerate(seasonal_popular[:top_n])
        ]
        return recommendations

    # Remaining code for collaborative and content-based filtering follows...


## Rank Score

To make the rank scores more meaningfully distributed across a wider range (e.g., between 0 and 1 with more significant variations), we can use a different scaling approach. Here are some options:

- Linear Scaling with a Larger Adjustment: Instead of using the formula directly, we can scale the index values differently to spread them more evenly.

- Exponential Decay: Use an exponential decay function to give a sharper difference between higher and lower-ranked products.
    + The rank score for each product is calculated using an exponential decay function (decay_factor ** idx). The decay_factor is set to 0.9, which means the scores decrease exponentially with increasing rank.
    + This approach ensures that products ranked higher receive significantly higher scores, and the scores decrease faster compared to linear scaling.
    + Adjust Decay Factor: You can adjust the decay_factor to control the rate of decrease. A value closer to 1 (e.g., 0.95) will result in a slower decrease, while a smaller value (e.g., 0.7) will result in a faster decrease in scores.

In [76]:
# Example usage
user_id = 88813  # Replace with the actual user ID
get_recommendations(user_id, top_n=10)

No liked products found for user 88813. Returning popular seasonal products.


[{'customer_id': 88813, 'product_id': 6461, 'rank_score': 1.0},
 {'customer_id': 88813, 'product_id': 39386, 'rank_score': 0.9},
 {'customer_id': 88813, 'product_id': 53759, 'rank_score': 0.81},
 {'customer_id': 88813, 'product_id': 1855, 'rank_score': 0.7290000000000001},
 {'customer_id': 88813, 'product_id': 24442, 'rank_score': 0.6561},
 {'customer_id': 88813, 'product_id': 40107, 'rank_score': 0.5904900000000001},
 {'customer_id': 88813, 'product_id': 3577, 'rank_score': 0.531441},
 {'customer_id': 88813, 'product_id': 42650, 'rank_score': 0.4782969000000001},
 {'customer_id': 88813, 'product_id': 30270, 'rank_score': 0.4304672100000001},
 {'customer_id': 88813, 'product_id': 54735, 'rank_score': 0.3874204890000001}]

## Generate 4 CSV files

In [88]:
import numpy as np
import pandas as pd
import ast
from scipy.sparse import load_npz

# Assuming customer_df, product_df, similarity_df, and user_similarity_matrix_sparse are already defined.

# Load the user similarity matrix
user_similarity_matrix_sparse = load_npz('user_similarity_matrix.npz')
user_similarity_df = pd.DataFrame(user_similarity_matrix_sparse.toarray(), index=customer_df['customer_id'], columns=customer_df['customer_id'])

# Function to determine the current season
# We'll iterate over each of the four seasons: 'Spring', 'Summer', 'Fall', 'Winter'
def determine_season(season):
    return season

# Function to get similar users for a given user_id
def get_similar_users(user_id, top_n=5):
    if user_id in user_similarity_df.index:
        similar_users = user_similarity_df.loc[user_id].sort_values(ascending=False).drop(user_id).head(top_n).index.tolist()
        return similar_users
    else:
        return []

# Function to get hybrid recommendations for a user
def get_recommendations(user_id, current_season, top_n=10):
    recommendations = []

    # Get the user's liked products
    user_data = customer_df[customer_df['customer_id'] == user_id]
    if user_data.empty:
        return recommendations  # Return empty if user not found

    liked_products = user_data['past_trans'].values[0]
    liked_products = ast.literal_eval(liked_products) if isinstance(liked_products, str) else liked_products

    # If no liked products are found, provide a default list of popular seasonal products
    if not liked_products:
        seasonal_popular = product_df[product_df['season'] == current_season]['id'].value_counts().index.tolist()
        total_products = len(seasonal_popular)
        decay_factor = 0.9  # Adjust decay factor to control how fast the score decreases
        recommendations = [
            {
                'customer_id': user_id,
                'product_id': product,
                'rank_score': decay_factor ** idx  # Exponentially decaying score
            }
            for idx, product in enumerate(seasonal_popular[:top_n])
        ]
        return recommendations

    # Get similar users
    similar_users = get_similar_users(user_id, top_n=5)
    similar_users_liked_products = []
    similar_users_scores = []

    if similar_users:
        # Get products liked by similar users and their corresponding similarity scores
        for similar_user in similar_users:
            similarity_score = user_similarity_df.loc[user_id, similar_user]
            similar_user_products = customer_df[customer_df['customer_id'] == similar_user]['past_trans'].values[0]
            similar_user_products = ast.literal_eval(similar_user_products) if isinstance(similar_user_products, str) else similar_user_products
            similar_users_liked_products.extend(similar_user_products)
            similar_users_scores.extend([similarity_score] * len(similar_user_products))

    # Get products similar to what the user liked
    similar_products = similarity_df[similarity_df['product_id'].isin(liked_products)]
    similar_products_ids = similar_products['similar_product_id'].tolist()
    similar_products_scores = similar_products['distance'].tolist()  # Assuming smaller distance means higher similarity

    # Filter recommendations by season
    seasonal_products = product_df[product_df['id'].isin(similar_products_ids + similar_users_liked_products) &
                                   (product_df['season'] == current_season)]

    if seasonal_products.empty:
        seasonal_products = product_df[product_df['season'] == current_season].head(top_n)

    # Aggregate scores for products
    product_scores = {}

    # Add scores from similar users' liked products
    for product_id, score in zip(similar_users_liked_products, similar_users_scores):
        if product_id in seasonal_products['id'].values:
            product_scores[product_id] = product_scores.get(product_id, 0) + score

    # Add scores from similar products
    for product_id, score in zip(similar_products_ids, similar_products_scores):
        if product_id in seasonal_products['id'].values:
            similarity_score = 1 / (1 + score)  # Using inverse distance as a similarity score
            product_scores[product_id] = product_scores.get(product_id, 0) + similarity_score

    # Normalize the scores to be between 0 and 1
    max_score = max(product_scores.values()) if product_scores else 1
    for product_id in product_scores:
        product_scores[product_id] /= max_score

    # Sort products by their aggregated scores in descending order and select the top N
    ranked_products = sorted(product_scores.items(), key=lambda x: x[1], reverse=True)[:top_n]

    # Prepare the recommendation list for the user
    recommendations = [{'customer_id': user_id, 'product_id': product_id, 'rank_score': rank_score}
                       for product_id, rank_score in ranked_products]

    return recommendations




In [ ]:
# Generate recommendations for all customers for each season
seasons = ['Spring']

for season in seasons:
    all_recommendations = []
    for user_id in customer_df['customer_id']:
        user_recommendations = get_recommendations(user_id, current_season=season, top_n=10)
        all_recommendations.extend(user_recommendations)

    # Convert all recommendations to DataFrame and save to CSV for the current season
    recommendations_df = pd.DataFrame(all_recommendations)
    csv_filename = f'recommendations_{season}.csv'
    recommendations_df.to_csv(csv_filename, index=False)
    print(f"Recommendations for season '{season}' saved to {csv_filename}.")

output for LLM Integration
